In [ ]:
%matplotlib inline

from os.path import join as pjoin
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from datetime import datetime

from matplotlib import patheffects
pe = patheffects.withStroke(foreground="white", linewidth=3)
matplotlib.rcParams['font.sans-serif'] = 'Arial'
sns.set_context("notebook")

In [ ]:
BASEDIR = r"..\data"
DATADIR = r"..\data\allevents"
eventFile = pjoin(DATADIR, "results", "stormclass.csv") # Change to all classified storms

stormdf = pd.read_csv(eventFile, usecols=[1, 2, 3],
                      parse_dates=['date'],
                      dtype={
                          'stnNum': int,
                          'stormType': 'category'
                          })

In [ ]:
# Change to all stations:
stndf = pd.read_csv(pjoin("../data", 'StationDetails.txt'), index_col="stnNum")

def loadData(stnNum):
    fname = pjoin(DATADIR, "events", f"{stnNum:06d}.pkl")
    df = pd.read_pickle(fname)
    df['date'] = pd.to_datetime(df['date'])
    df['stnNum'] = stnNum
    df.reset_index(inplace=True)
    # Create a MultiIndex based on station number and storm date:
    df.set_index(['stnNum', 'date'], inplace=True)
    return df

# Discard - will only load the data for the events as needed
#dflist = []
#for stn in stndf.index:
#    df = loadData(stn)
#    dflist.append(df)

#alldf = pd.concat(dflist)
#alldf['idx'] = alldf.index

In [ ]:
def plotEvent(df, ax, wlims=(0, 100), tlims=(-4, 4), plims=(-2, 2)):
    """
    Plot a time series onto an axis

    :param df: DataFrame containing time series of temperature, wind speed, dew point and station pressure values
    :param ax: `matplotlib.Axes`
    """
    axt = ax.twinx()
    axp = ax.twinx()
    ax.set_zorder(1)
    ax.patch.set_visible(False)
    l1 = axt.plot(df.tdiff, df.tempanom, label=r"Temperature [$^o$C]",
             color='r', marker='^', markerfacecolor="None", lw=1, path_effects=[pe], zorder=1,
             markevery=5)
    l2 = axt.plot(df.tdiff, df.dpanom, label=r"Dew point [$^o$C]", color='orangered', marker='.', markerfacecolor="None",
             lw=1, path_effects=[pe], zorder=1, markevery=5)

    l3 = ax.plot(df.tdiff, df.windgust, label="Wind gust [km/h]", color="#4C72B0",
            lw=2, path_effects=[pe], markerfacecolor="None",zorder=100)
    l4 = axp.plot(df.tdiff, df.stnpanom, label="Pressure [hPa]", color='purple', lw=2, path_effects=[pe],
             ls='--',)


    #axt.spines['right'].set_position(("axes", 1.075))
    axt.spines[['right']].set_color('r')
    axt.yaxis.label.set_color('r')
    axt.tick_params(axis='y', colors='r')
    axt.set_ylabel(r"Temperature anomaly [$^o$C]")

    ax.set_ylabel("Gust wind speed [km/h]")
    # ax.set_xlabel("Time since maximum gust [minutes]")

    axp.spines[['right']].set_position(('axes', 1.075))
    axp.spines[['right']].set_color('purple')
    axp.yaxis.label.set_color('purple')
    axp.tick_params(axis='y', colors='purple')
    axp.set_ylabel("Pressure anomaly [hPa]")

    gmin, gmax = ax.get_ylim()
    pmin, pmax = axp.get_ylim()
    tmin, tmax = axt.get_ylim()
    ax.set_ylim(wlims)
    ax.set_xlim((-60, 60))
    axp.set_ylim(plims)
    axt.set_ylim(tlims)
    #axt.set_ylim((min(-2.0, tmin), max(tmax, 2.0)))
    #ax2.set_ylim((0, 360))
    #ax2.set_yticks(np.arange(0, 361, 90))
    #axr.set_ylim((0, 100))
    #ax.set_title(meants.index[0])
    ax.grid(True)
    #ax2.grid(False)
    axt.grid(False)
    axp.grid(False)
    #axr.grid(False)

def identify_axes(axdict, labels):
    kw = dict(ha="left", va="top", fontsize=12, color='black',
              bbox=dict(boxstyle='square',
                        ec="white",
                        fc="white",
                        alpha=0.7))
    for (k, ax), label in zip(axdict.items(), labels):
        ax.text(.02, 0.95, f"{k} {label}", transform=ax.transAxes, **kw)

In [ ]:
# This is where we load the data and then filter on location/date:

# Synoptic storm - Double Island Point
syndata = loadData(40068)
syndf = syndata[syndata.index==(40068, datetime(2010, 10, 9))].rename({"datetime":"tdiff"})

# Synoptic front - Longreach
syfdata = loadData(36031)
syfdf = syfdata[syfdata.index==(36031, datetime(2018, 12, 12))].rename({"datetime":"tdiff"})

# Storm burst - Hamilton Island
stbdata = loadData(33106)
stbdf = stbdata[stbdata.index==(33106, datetime(2014, 1, 29))].rename({"datetime":"tdiff"})

# Thunderstorm - Blackwater Airport
tstdata = loadData(35134)
tstdf = tstdata[tstdata.index==(35134, datetime(2018, 10, 11))].rename({"datetime":"tdiff"})

# Front up - Winton Airport
frudata = loadData(37039)
frudf = frudata[frudata.index==(37039, datetime(2016, 11, 11))].rename({"datetime":"tdiff"})

# Front down - Townsville
frddata = loadData(32040)
frddf = frddata[frddata.index==(32040, datetime(2015, 1, 19))].rename({"datetime":"tdiff"})

# Spike - Birdsville
spkdata = loadData(38026)
spkdf = spkdata[spkdata.index==(38026, datetime(2013, 4, 17))].rename({"datetime":"tdiff"})

In [ ]:
fig, axes = plt.subplot_mosaic(
    [["(a)", "(b)"],
     ["(c)", "(d)"],
     ["(e)", "(f)"]],
    sharex=True, sharey=True,
    figsize=(20, 12)
)
plotEvent(syndf, axes['(a)'], wlims=(0, 120), tlims=(-6, 6), plims=(-4, 4))
plotEvent(syfdf, axes['(b)'], wlims=(0, 120), tlims=(-6, 6), plims=(-4, 4))
plotEvent(stbdf, axes['(c)'], wlims=(0, 120), tlims=(-6, 6), plims=(-4, 4))
plotEvent(tstdf, axes['(d)'], wlims=(0, 120), tlims=(-6, 6), plims=(-4, 4))
plotEvent(frudf, axes['(e)'], wlims=(0, 120), tlims=(-6, 6), plims=(-4, 4))
plotEvent(frddf, axes['(f)'], wlims=(0, 120), tlims=(-6, 6), plims=(-4, 4))

identify_axes(axes, labels=
              ["Synoptic storm\nDouble Island Point",
               "Synoptic front\nCape Moreton",
               "Storm-burst\nHamilton Is",
               "Thunderstorm\nBlackwater",
               "Front-up\nWinton",
               "Front-down\nTownsville"])
tlegendline = mlines.Line2D([], [], color='red', marker='^',
                            markerfacecolor="None",
                            label=r"Temperature [$^o$C]")
dlegendline = mlines.Line2D([], [], color='orangered',
                            marker='.', markerfacecolor="None",
                            label=r"Dew point [$^o$C]")
plegendline = mlines.Line2D([], [], color='purple',
                            ls='--', label=r"Pressure [hPa]")
wlegendline = mlines.Line2D([], [], color='#4C72B0',
                            label=r"Wind speed [km/h]")
axes['(a)'].legend(handles=[wlegendline, plegendline,
                            tlegendline, dlegendline], loc=3,
                   fontsize='small')
axes['(e)'].set_xlabel("Time since maximum gust [min]")
axes['(f)'].set_xlabel("Time since maximum gust [min]")
plt.tight_layout()

#fig.savefig(pjoin(BASEDIR, "plots", "example_visual_storm_profile.png"),
#            bbox_inches="tight")